# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','LateLesion.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

---

plotting group bar plots

In [ ]:
def plot_event_rec(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nPre=3, nPost=3, nRec=3, 
                   seed=1, dot_plot=True, **kwarg):

    if badAnimals is None:
        badAnimals=[]
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-diff,x_c,x_c+diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    _,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)

    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values()))
    
    xPre= np.nanmedian(data[:,nSessionPre-nPre:nSessionPre],axis=1)
    xPost=np.nanmedian(data[:,nSessionPre:nPost+nSessionPre],axis=1)
    xRec=[]
    for i in range(data.shape[0]):
        d=data[i,:]
        try:
            ind=np.where(~np.isnan(d) == True)[0][-1]
        except IndexError:
            xRec.appen(np.nan)
            continue
        xRec.append(np.nanmedian(d[ind-nRec:]))
    xRec=np.array(xRec)
    
    
    x=np.vstack([xPre,xPost,xRec]).T
    
    y=np.nanpercentile(x,50,axis=0)
    yerr=np.nanpercentile(x,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='o', zorder=5, ms=2, elinewidth=1, **kwarg)
    
    #plotting individual animals
    if dot_plot:
        np.random.seed(seed=seed)
        _coeff=5
        for i in range(xPre.shape[0]):
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitRec =np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)

            ax.scatter([jitPre,jitPost,jitRec],[xPre[i],xPost[i],xRec[i]],s=.2,c='gray', marker='o', alpha=.8)

    
    locs=ax.xaxis.get_ticklocs()
    locs=[*locs.tolist(),*list(x_pos)]
    labels=ax.xaxis.get_ticklabels()
    labels.extend(['before','after','final'])
    ax.set_xticks(locs)
    ax.set_xticklabels(labels)
    ax.xaxis.set_tick_params(rotation=-45)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0,100])
    ax.set_ylabel(TaskParamToPlot)
    ax.set_xlabel('Sessions relative to break / lesion')

    return xPre,xPost,xRec

def add_legend_for_lesion_types(ax,N):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='tomato', marker='o',
                                       markerfacecolor='tomato', markersize=3, label=f'DLS\n$n={N[1]}$')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:blue', marker='o',
                                       markerfacecolor='xkcd:blue', markersize=3, label=f'DMS\n$n={N[2]}$')
    m_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='magenta', marker='o',
                                       markerfacecolor='magenta', markersize=3, label=f'DS\n$n={N[3]}$')
    k_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='k', marker='o',
                                       markerfacecolor='k', markersize=3, label=f'Control\n$n={N[0]}$')

#     errorbar = matplotlib.lines.Line2D([0,0], [0,1], color='k', label='95% CI')


    leg=ax.legend(handles=[k_marker,r_marker,b_marker,m_marker],loc=(0,.9),mode='expand',ncol=4,
                  facecolor=None,edgecolor=None, fontsize='xx-small',frameon=False)
    return leg

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot="% good trials"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)
    ax.set_xticks([])
    plot_event_rec(root, ax, Profiles,x_pos=(0,1,2),color='k')
    ax.set_xlim([-1,10])

---

Plot the speed definition graphic

In [ ]:
def plot_speed_def(ax):
    x0=np.arange(0,7,.01)
    y0=10*x0+10

    x1=np.arange(7, 8.16,.01)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)

    y=smooth(y*1.1,sigma=50,mode='constant')

    x40=np.where(np.logical_and(y<=40,x>7))[0][0]
    x60=np.where(np.logical_and(y<=60,x>7))[0][0]
    #plot the traj
    ax.plot(x, y, lw=2, c='navy', )
    #plot the 40 and 60 horizontal lines
    ax.hlines(y=40, xmin=-10, xmax=x[x40], lw=ax.spines['bottom'].get_lw(), ls='--')
    ax.hlines(y=60, xmin=-10, xmax=x[x60],lw=ax.spines['bottom'].get_lw(), ls='--')
    #plot the magenta piece
    ax.plot([x[x40],x[x60]],[40,60], lw=ax.spines['bottom'].get_lw(), c='m',zorder=4)
    #plot the x-axis at the magenta piece
    ax.arrow(x[x60],60,1.3,0,lw=ax.spines['bottom'].get_lw(),head_width=2,head_length=.2, ec='m', fc='m',zorder=5)
    # plot the curve
    x2=7.5
    y2=y[np.where(x>=x2)[0][0]]
    ax.add_patch(FancyArrowPatch(posA=(x[x60]+.8,62), posB=(x2-.1,y2-2),
                                 arrowstyle='-', connectionstyle='arc3,rad=-.5',color='m',
                                 mutation_scale=15,lw=ax.spines['bottom'].get_lw(),zorder=6))
    #writing 
    ax.text(7.9,40,'speed',rotation=-75, c='m', fontsize='xx-small',ha='left',va='center')
    
    
    ax.set_xticks([0,1,2,3,4,5,6,7,8])
    ax.set_yticks(range(0,91,10))
    ax.spines['bottom'].set_bounds(0,8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.1,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)')
    ax.set_xlabel('Trial time (s)')


In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    plot_speed_def(ax)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='black'
    DLSColor='tomato'
    DMSColor='xkcd:blue'
    DSColor='magenta'
    
       

    
    #===============================================
    
    # GRID 1 PARAMS
    
    TaskParamToPlot1="% good trials" 
    
    profileCtrlPre1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profileCtrlPost1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }  
    ProfilesCtrl1=(profileCtrlPre1,profileCtrlPost1,)
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS1=(profilePreLesion1,profileDLS1,)
    ProfilesDMS1=(profilePreLesion1,profileDMS1,)
    ProfilesDS1 =(profilePreLesion1,profileDS1,)
    
    #================================================
    
    # GRID 2 PARAMS
    
    TaskParamToPlot2="Motor Sequence Score"
    

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(7,5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
    
    ##########################################
    # 1: % correct trials
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0.02, bottom=0.63, right=0.45, top=.99)
    ax1= fig.add_subplot(gs1[0])
    
    ax1.set_xticks([])
    #DLS
    xDLS1,*_=plot_event_rec(root, ax1, ProfilesDLS1,x_pos=(4,5,6),color=DLSColor)

    #DMS
    xDMS1,*_=plot_event_rec(root, ax1, ProfilesDMS1,x_pos=(8,9,10),color=DMSColor)

    #DS
    xDS1,*_ =plot_event_rec(root, ax1, ProfilesDS1,x_pos=(12,13,14),color=DSColor)

    #Control
    xCtrl1,*_=plot_event_rec(root, ax1, ProfilesCtrl1,x_pos=(0,1,2),color=CtrlColor)
    
    add_legend_for_lesion_types(ax1,(len(xCtrl1),len(xDLS1),len(xDMS1),len(xDS1)))
    
    ax1.set_ylim([-1,101])
    ax1.spines['left'].set_bounds(0,100)
    ax1.set_xlim([-1,15])
    ax1.set_ylabel('% correct trials')

    
    
    ##########################################
    # 2: % sequential score
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=0.55, bottom=0.63, right=0.99, top=.99)
    ax2= fig.add_subplot(gs2[0])
    
    ax2.set_xticks([])
    #DLS
    plot_event_rec(root, ax2, ProfilesDLS1,TaskParamToPlot=TaskParamToPlot2,x_pos=(4,5,6),color=DLSColor)

    #DMS
    plot_event_rec(root, ax2, ProfilesDMS1,TaskParamToPlot=TaskParamToPlot2,x_pos=(8,9,10),color=DMSColor)

    #DS
    plot_event_rec(root, ax2, ProfilesDS1,TaskParamToPlot=TaskParamToPlot2,x_pos=(12,13,14),color=DSColor)

    #Control
    plot_event_rec(root, ax2, ProfilesCtrl1,TaskParamToPlot=TaskParamToPlot2,x_pos=(0,1,2),color=CtrlColor, dot_plot=True)
    
    
    ax2.set_xlim([-1,15])
    ax2.set_ylim([-.01,1.01])
    ax2.spines['left'].set_bounds(0,1)
    ax2.set_yticks([0,.2,.4,.6,.8,1])
    ax2.set_yticklabels([0,20,40,60,80,100])
    ax2.set_ylabel('% Routine')
    

    ##########################################
    # 3: speed def
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=0.02, bottom=0.3, right=0.2, top=.5)
    ax3= fig.add_subplot(gs3[0])
    
    plot_speed_def(ax3)

    
    
    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LateLesion.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()